### Reward in all, easy, and hard sessions
- Comparing Unstruc and Struc where the difference in probability is more than 0.4 

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess
trial_filter = dict(min_trials=100, clip_max=100)

rwd_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(**trial_filter)
    mab_easy = task.filter_by_deltaprob(delta_min=0.4)
    mab_hard = task.filter_by_deltaprob(delta_min=0, delta_max=0.35)

    rp = task.get_reward_probability()
    rp_easy = mab_easy.get_reward_probability()
    rp_hard = mab_hard.get_reward_probability()

    df = pd.DataFrame(
        dict(
            trial_id=np.arange(rp.size) + 1,
            rp=rp,
            rp_easy=rp_easy,
            rp_hard=rp_hard,
            name=exp.sub_name,
            grp="struc" if task.is_structured else "unstruc",
            first_experience=True if "Exp1" in exp.sub_name else False,
        )
    )
    rwd_df.append(df)

rwd_df = pd.concat(rwd_df, ignore_index=True)
mab_subjects.GroupData().save(rwd_df, "reward_prob_difficulty_level")

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from neuropy import plotting
import mab_subjects
import numpy as np
from statplotannot.plots.colormaps import colors_mab
from statplotannot.plots import SeabornPlotter

fig = plotting.Fig(6, 3, num=1, fontsize=10)

grpdata = mab_subjects.GroupData()
df = grpdata.reward_prob_difficulty_level
df = df[df["first_experience"] == True]

ax = fig.subplot(fig.gs[0])
hue_order = ["unstruc", "struc"]
linestyles = ["-", "--", ":"]

for i, y in enumerate(["rp", "rp_easy", "rp_hard"]):

    plot_kw = dict(data=df, x="trial_id", y=y, hue="grp", hue_order=hue_order, ax=ax)
    sns.lineplot(
        palette=colors_mab(),
        # palette=["#E89317", "#176CE8"],
        # palette=["#d58915", "#1660d0"],
        lw=1.1,
        # edgecolor="white",
        # facecolor=(0, 0, 0, 0),
        # alpha=0.4,
        ls=linestyles[i],
        err_kws=dict(edgecolor="none"),
        errorbar="se",
        **plot_kw,
    )
SeabornPlotter.fix_legend(ax)

ax.set_title(r"Performance in all sessions")
ax.set_ylabel("Performance")
# ax.set_ylim(0.45, 1)
ax.set_xticks([1, 50, 100])
sns.move_legend(ax, "upper left", bbox_to_anchor=(1, 1))

# ax.legend("")

### Reward rate matrix
- Bins are represented by probability combination

In [ ]:
import numpy as np
import pandas as pd
import mab_subjects
from scipy.ndimage import gaussian_filter1d

exps = mab_subjects.unstruc.allsess + mab_subjects.struc.allsess

prob_perf_df = []

for i, exp in enumerate(exps):
    print(exp.sub_name)
    task = exp.b2a.filter_by_trials(min_trials=100, clip_max=100)

    probs = np.unique(task.probs[task.is_session_start.astype(bool)], axis=0)

    arr = []
    for prob in probs:
        r_curve = task.filter_by_probs(prob).get_reward_probability()
        r_rate = np.mean(r_curve[-10:])
        # prob_perf = gaussian_filter1d(prob_perf, sigma=2)
        arr.append([prob[0], prob[1], r_rate])

    df = pd.DataFrame(np.array(arr), columns=["prob1", "prob2", "delta_improvement"])
    df["name"] = exp.sub_name
    df["grp"] = "struc" if task.is_structured else "unstruc"
    df["first_experience"] = True if "Exp1" in exp.sub_name else False

    prob_perf_df.append(df)

prob_perf_df = pd.concat(prob_perf_df, ignore_index=True)
# mab_subjects.GroupData().save(perf_df, "perf_100min150max_10bin")

In [ ]:
import matplotlib.pyplot as plt
from neuropy.plotting import Fig
import seaborn as sns
from scipy.stats import binned_statistic_2d
from statplotannot.plots.colormaps import colors_mab

fig = Fig(1, 3, size=(11, 3), num=1)

diag = []
diag_error = []
for g, grp in enumerate(["unstruc", "struc"]):
    df = prob_perf_df[prob_perf_df["grp"] == grp]
    x, y, values = df["prob1"], df["prob2"], df["delta_improvement"]
    bins = np.linspace(0, 0.9, 10) + 0.05
    centers = (bins[:-1] + bins[1:]) / 2
    centers = np.round(centers, 2)

    H, xedges, yedges, _ = binned_statistic_2d(
        x, y, values, statistic=np.nanmean, bins=bins
    )
    H_error = binned_statistic_2d(x, y, values, statistic=np.nanstd, bins=bins)[0]
    H_n = binned_statistic_2d(x, y, values, statistic="count", bins=bins)[0]

    diag.append(np.diag(np.rot90(H)))
    diag_error.append(np.diag(np.rot90(H)) / np.sqrt(np.diag(np.rot90(H_n))))

    ax = fig.subplot(fig.gs[g])
    # sns.heatmap(H)
    ax.pcolormesh(H, cmap="plasma", shading="nearest", vmin=0.1, vmax=0.9)
    ax.set_xticks(np.arange(len(centers)), centers)
    ax.set_yticks(np.arange(len(centers)), centers)


ax = fig.subplot(fig.gs[2])
for i, arr in enumerate(diag):
    ax.errorbar(
        np.arange(len(arr)) + np.random.randn(len(arr)) * 0.2,
        arr,
        yerr=diag_error[i],
        label="unstruc",
        color=colors_mab()[i],
        ls="",
        marker="o",
        lw=2,
    )